In [26]:
from __future__ import division

import re
import sys
import os

import random
import math

import numpy as np
import pandas as pd
import scipy

import gensim, logging

import pymorphy2
from nltk import word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from stop_words import get_stop_words

from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

morph = pymorphy2.MorphAnalyzer()
tokenizer = TreebankWordTokenizer()

RUS_LETTERS = u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

2017-03-23 02:32:42,681 : INFO : Loading dictionaries from /usr/local/lib/python3.5/site-packages/pymorphy2_dicts/data
2017-03-23 02:32:43,079 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


Сначала нужно провести предобработку текстов: лемматизировать, удалить стоп-слова, возможно, разметить части речи (но последнее необязательно).

In [27]:
anekdots = [f for f in os.listdir('./weka/anekdots/') if f.endswith('.txt')]

In [28]:
izvest = [f for f in os.listdir('./weka/izvest/') if f.endswith('.txt')]

In [29]:
teh_mol = [f for f in os.listdir('./weka/teh_mol/') if f.endswith('.txt')]

In [30]:
len(anekdots)

125

In [31]:
m = 'ruscorpora_1_300_10.bin'
if m.endswith('.vec'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.Word2Vec.load(m)

2017-03-23 02:32:48,225 : INFO : loading projection weights from ruscorpora_1_300_10.bin
2017-03-23 02:32:54,408 : INFO : loaded (184973, 300) matrix from ruscorpora_1_300_10.bin


In [32]:
arr_corpora = []

for a in anekdots:
    with open('./weka/anekdots/' + a, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = 'anekdots'
        dicti['text'] = text
        arr_corpora.append(dicti)

for i in izvest:
    with open('./weka/izvest/' + i, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = 'izvest'
        dicti['text'] = text
        arr_corpora.append(dicti)
        
for t in teh_mol:
    with open('./weka/teh_mol/' + t, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = 'teh_mol'
        dicti['text'] = text
        arr_corpora.append(dicti)
        
        
random.shuffle(arr_corpora)  # перемешиваем данные
    
df = pd.DataFrame(arr_corpora)  # создаем датасет

In [33]:
data_train, data_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [34]:
df.head()

,class,text
0,teh_mol,Однажды...\nИ хватит об этом!\nКак-то раз берл...
1,teh_mol,Сергей Алексеев. Тропы еще в антимир не протоп...
2,izvest,Лесков Сергей. Калмыкию оставили без космодром...
3,anekdots,"Sех, он и она\n***\nНа мосту стояли трое: он, ..."
4,izvest,Анненков Андрей. Горячие кристаллы на остывающ...


In [35]:
def do_smth_with_model(steps):
    print('\nModel train')
    pipeline = Pipeline(steps=steps)

    cv_results = cross_val_score(pipeline,
                                 data_train,
                                 class_train,
                                 cv=10,
                                 scoring='accuracy',
                                )
    print(cv_results.mean(), cv_results.std())

    pipeline.fit(data_train, class_train)
    class_predicted = pipeline.predict(data_test)
    print(class_predicted)

    print(classification_report(class_test, class_predicted ))

    return pipeline, class_predicted

In [36]:
transit = {'ADJF':'ADJ',
'ADJS' : 'ADJ',
'ADVB' : 'ADV',
'COMP' : 'ADV',
'CONJ' : 'CCONJ',
'GRND' : 'VERB',
'INFN' : 'VERB',
'INTJ' : 'INTJ',
'LATN' : 'X',
'NOUN' : 'NOUN',
'NPRO' : 'PRON',
'NUMB' : 'NUM',
'NUMR' : 'NUM',
'PNCT' : 'PUNCT' ,
'PRCL' : 'PART',
'PRED' : 'ADV',
'PREP' : 'ADP',
'PRTF' : 'ADJ',
'PRTS' : 'VERB',
'ROMN' : 'X',
'SYMB' : 'SYM',
'UNKN' : 'X',
'VERB' : 'VERB'}

robj = re.compile('|'.join(transit.keys()))

def cleanization(text):
    for line in text:
        # 1. Все буквы в нижний регистр
        text_text = text.lower()

        # 2. Удаление всех небукв
        letters_only = ''
        for _c in text_text:
            if _c in RUS_LETTERS:
                letters_only += _c
            else:
                letters_only += ' '

        # 3. Заменяем множественные пробелы
        while '  ' in letters_only:
            letters_only = letters_only.replace('  ', ' ')

        # 4. Токенизация
        word_list = tokenizer.tokenize(letters_only)

        # 5. Лемматизация
        clean_word_list = [morph.parse(word)[0].normal_form for word in word_list]  # лемматизация
    
        # 6. * Удаление стоп-слов + добавление тегов - части речи
        # meaningful_words = [word for word in clean_word_list if word not in get_stop_words('ru')] # стоп-слова
        meaningful_words = [str(word) + '_' + robj.sub(lambda m: transit[m.group(0)], str(morph.parse(word)[0].tag.POS)) for word in clean_word_list]
        return ' '.join(meaningful_words) 

In [37]:
from __future__ import division
def mean(a):
    return sum(a) / len(a)

In [38]:
def word2vec_mean(text):
    """Сколько чисел."""
    arr = []
    clean_text = cleanization(text)
    # для каждого слова в тексте выводим его вектор
    for word in clean_text.split(' '):
    # есть ли слово в модели? Может быть, и нет
        if word in model:
            arr.append(model[word])
    if len(list(map(mean, zip(*arr)))) != 0:
        return list(map(mean, zip(*arr)))
    else:
        return [0 for i in range(0, 300)]



class FunctionFeaturizer(TransformerMixin):
    """ Для создания своего вектора я использовала вектор слова в модели word2vec"""
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        fvs = []
        for datum in X:
            fv = word2vec_mean(datum)
            fvs.append(fv)
        return np.array(fvs)

In [39]:
w2v_featurizer = FunctionFeaturizer()  # создание своего векторизатора

In [21]:
# numbers('Для создания своего вектора я использовала несколько фич: длину текста, количество заглавных букв')

In [22]:
# Свой векторизатор
print('\nCustom Transformer')
lg_pipeline, label_predicted = do_smth_with_model(steps=[('custom', w2v_featurizer),
                                                      ('classifier', LogisticRegression())])


Custom Transformer

Model train
0.88672877271 0.0263862200298
['izvest' 'anekdots' 'izvest' 'anekdots' 'teh_mol' 'anekdots' 'teh_mol'
 'anekdots' 'anekdots' 'izvest' 'anekdots' 'anekdots' 'izvest' 'izvest'
 'anekdots' 'izvest' 'teh_mol' 'izvest' 'teh_mol' 'teh_mol' 'anekdots'
 'teh_mol' 'teh_mol' 'anekdots' 'anekdots' 'teh_mol' 'anekdots' 'anekdots'
 'anekdots' 'teh_mol' 'anekdots' 'teh_mol' 'izvest' 'teh_mol' 'teh_mol'
 'anekdots' 'izvest' 'teh_mol' 'izvest' 'teh_mol' 'izvest' 'anekdots'
 'izvest' 'izvest' 'izvest' 'teh_mol' 'anekdots' 'teh_mol' 'anekdots'
 'izvest' 'anekdots' 'teh_mol' 'anekdots' 'teh_mol' 'izvest' 'anekdots'
 'anekdots' 'anekdots' 'anekdots' 'anekdots' 'teh_mol' 'izvest' 'teh_mol'
 'izvest' 'teh_mol' 'anekdots' 'anekdots' 'teh_mol' 'izvest' 'izvest'
 'anekdots' 'izvest' 'izvest' 'izvest' 'anekdots']
             precision    recall  f1-score   support

   anekdots       0.97      0.97      0.97        30
     izvest       0.87      0.83      0.85        24
    teh_

In [42]:
# Проверка работы модели на наших тестовых коллокациях
def predictor(collocations_array, pipeline):
    arr = []
    df1 = pd.DataFrame({'text': collocations_array})
    for i in df1.text:
        arr.append(i)
    с = 0
    for i in pipeline.predict(df1.text):
        print(arr[с], ':', i)
        с += 1


collocations_array = ['История произошла перед новым годом. Иду в Ашан закупаться, а за мной по тротуару следом два парня идут. И довольно громко обсуждают своего друга, причем довольно явно слышится кавказский акцент и характерные словечки, вроде "брат" при каждом обращении. Парни молодые, речь вдобавок с "пацанскими" оборотами. И тут звучит фраза, которую я совершенно не ожидал услышать: "Брат, я ему и говорю - что за врач из тебя будет, если ты на нейроанатомию забиваешь?!" И как-то тепло на душе стало, что это не просто пацан, а человек, уже понимающий ценность образования.', 
                      '- А ты смогла бы всю жизнь прожить с одним мужчиной?- Да я-то смогла бы. Но мужика жалко.']
predictor(collocations_array, lg_pipeline)

История произошла перед новым годом. Иду в Ашан закупаться, а за мной по тротуару следом два парня идут. И довольно громко обсуждают своего друга, причем довольно явно слышится кавказский акцент и характерные словечки, вроде "брат" при каждом обращении. Парни молодые, речь вдобавок с "пацанскими" оборотами. И тут звучит фраза, которую я совершенно не ожидал услышать: "Брат, я ему и говорю - что за врач из тебя будет, если ты на нейроанатомию забиваешь?!" И как-то тепло на душе стало, что это не просто пацан, а человек, уже понимающий ценность образования. : anekdots
- А ты смогла бы всю жизнь прожить с одним мужчиной?- Да я-то смогла бы. Но мужика жалко. : anekdots


In [ ]:
# Свой векторизатор
print('\nCustom Transformer')
lg2_pipeline, label_predicted = do_smth_with_model(steps=[('custom', w2v_featurizer),
                                                      ('classifier', LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=4))])


Custom Transformer

Model train
